In [1]:
!pwd

/root/autodl-tmp/generativeModel/disguish_learning/LR_001


In [2]:
import os

In [3]:
os.chdir('/root/autodl-tmp/generativeModel/disguish_learning/LR_001')

In [4]:
import transformers
from transformers import  AutoModelForCausalLM,AutoTokenizer

In [5]:


import torch
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
)
from datasets import load_dataset


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


In [6]:
cache_dir = '/root/autodl-tmp/model/'
model = AutoModelForCausalLM.from_pretrained('bigscience/bloom-3b',cache_dir=cache_dir,trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('bigscience/bloom-3b',cache_dir=cache_dir,trust_remote_code=True)

'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bigscience/bloom-3b/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f078c98c130>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/bigscience/bloom-3b/resolve/main/tokenizer_config.json


In [7]:
# make model as

In [8]:
model = prepare_model_for_int8_training(model)
lora_config = LoraConfig(
    r=8,#LORA_R,
    lora_alpha=16,#LORA_ALPHA,
    target_modules=["query_key_value"],#TARGET_MODULES,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model,lora_config)

/root/miniconda3/lib/python3.8/site-packages/peft/tuners/lora.py:180: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


In [9]:
data_cache_dir = '/root/autodl-tmp/data/'

In [10]:
data = load_dataset("cahya/instructions-zh",cache_dir=data_cache_dir)

Using custom data configuration cahya--instructions-zh-507237297bfcf9f5
Found cached dataset parquet (/root/autodl-tmp/data/cahya___parquet/cahya--instructions-zh-507237297bfcf9f5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
for x in data['train']:
    print(x)
    break

{'id': 42216, 'text': 'User: 虚假信息与虚假信息有什么区别?\nAssistant: 误导和误导之间有区别 - 误导是欺骗的使用来破坏沟通过程,而误导是错误的信息或事实的误解。 误导的一个常见例子是新闻来源故意报告虚假声明或误导性信息以传播宣传和影响意见。'}


In [12]:
def split_train_example(text:str):
    answer_prefix = "Assistant:"
    prompt_prefix = "User:"

    answer_start_idx = text.rfind(answer_prefix)
    if(answer_start_idx > 0):
        # this is an trian data
        answer = text[answer_start_idx + len(answer_prefix):]
        prompt = text[:(answer_start_idx + len(answer_prefix):)]#.replace(prompt_prefix,"")
    else:
        prompt = text
        answer = None

    return prompt,answer

In [13]:
def build_tokenzie_func(tokenizer,pad_idx=0,max_length=256,pad=True):
    def tokenize(example):
        text = example['text']
        prompt,answer = split_train_example(text)
        prompt_idxs = tokenizer(prompt)
        answer_idxs = tokenizer(answer) if answer is not None else []

        example = {}
        example['input_ids'] = prompt_idxs['input_ids'] + answer_idxs['input_ids']
        example['attention_mask'] = prompt_idxs['attention_mask'] + answer_idxs['attention_mask']
        example['labels'] = [0] * len(prompt_idxs['attention_mask']) + answer_idxs['input_ids']
        example['prompt'] = prompt
        example['answer'] = answer
        
        if(len(example['input_ids']) < max_length):
            count = max_length - len(example['input_ids'])
            example['input_ids'] = example['input_ids'] + [0] * count
            example['attention_mask'] = example['attention_mask']+ [0] * count
            example['labels'] = example['labels']+ [0] * count
        
        example['input_ids'] = example['input_ids'][:max_length]
        example['attention_mask'] = example['attention_mask'][:max_length]
        example['labels'] = example['labels'][:max_length]
        

        return example
    return tokenize
tokenize_func = build_tokenzie_func(tokenizer)

In [14]:
train_data = data['train'].map(tokenize_func)

Loading cached processed dataset at /root/autodl-tmp/data/cahya___parquet/cahya--instructions-zh-507237297bfcf9f5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-b67f8effbb24490f.arrow


In [15]:
print(train_data[0])

{'id': 42216, 'text': 'User: 虚假信息与虚假信息有什么区别?\nAssistant: 误导和误导之间有区别 - 误导是欺骗的使用来破坏沟通过程,而误导是错误的信息或事实的误解。 误导的一个常见例子是新闻来源故意报告虚假声明或误导性信息以传播宣传和影响意见。', 'input_ids': [210, 238785, 8401, 1961, 238785, 8401, 37242, 49076, 1021, 210, 19651, 5092, 745, 19651, 5092, 10040, 706, 49076, 647, 210, 19651, 5092, 644, 151711, 60697, 1572, 30949, 28543, 5425, 2818, 15, 1518, 19651, 5092, 644, 35892, 42287, 2140, 28866, 373, 201933, 420, 210, 19651, 5092, 16368, 39428, 40567, 644, 19312, 28349, 70251, 4840, 238785, 32460, 2140, 19651, 5092, 1963, 8401, 909, 35786, 26262, 745, 7962, 15344, 420, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [16]:
val_data = data['validation'].map(tokenize_func)
test_data = data['test'].map(tokenize_func)

Loading cached processed dataset at /root/autodl-tmp/data/cahya___parquet/cahya--instructions-zh-507237297bfcf9f5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-b436db858b02878b.arrow
Loading cached processed dataset at /root/autodl-tmp/data/cahya___parquet/cahya--instructions-zh-507237297bfcf9f5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-ff0a77ded71f3fb7.arrow


In [17]:
ddp = True if torch.cuda.device_count() > 1 else False
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=128//2,
        warmup_steps=100,
        num_train_epochs=3,
        learning_rate=1e-5,
        fp16=True,
        logging_steps=2000,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=2000,
        save_steps=2000,
        output_dir="lora-alpaca",
        save_total_limit=3,
        load_best_model_at_end=True,
        ddp_find_unused_parameters=False if ddp else None,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False

old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())
).__get__(model, type(model))


In [18]:
if torch.__version__ >= "2":
    model = torch.compile(model)

##############
# 训练循环

trainer.train()




print("\n If there's a warning about missing keys above, please disregard :)")

/root/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
2000,1.689000,0.728290
4000,0.695600,0.692487
6000,0.676100,0.679073
8000,0.662000,0.670674
10000,0.665600,0.664975
12000,0.657200,0.661186
14000,0.657600,0.657844
16000,0.652300,0.655650
18000,0.650000,0.653018
20000,0.650600,0.651126


KeyboardInterrupt: 

# 测试部分

In [19]:
prompt_idxs['input_ids']

NameError: name 'prompt_idxs' is not defined

In [20]:
prompt_idxs['input_ids'].min()

NameError: name 'prompt_idxs' is not defined

In [ ]:
model.disable_adapter()

In [21]:
for n,x in model.base_model.model.named_parameters():
    if("embedding" in n):
        print(x.size())

torch.Size([250880, 2560])
torch.Size([2560])
torch.Size([2560])


In [38]:
prompt = """
User:习近平是谁么
Assistant:
习近平是中国共产党总书记,也是中国国家主席。
User:他有什么职责
Assistant:习近平负责领导中国共产党和中国政府,并监督中国的政治和经济。
User:我们在聊谁呢？
"""
prompt_idxs = tokenizer(prompt)

prompt_idxs = {
    key : torch.LongTensor([prompt_idxs[key]]).cuda()
    for key in prompt_idxs
}


In [39]:

with torch.no_grad():
   x = model.generate(input_ids=prompt_idxs['input_ids'],max_new_tokens=56)


In [40]:
for z in x.cpu().numpy():   
    y= tokenizer.decode(z)
    print(y)


User:习近平是谁么
Assistant:
习近平是中国共产党总书记,也是中国国家主席。
User:他有什么职责
Assistant:习近平负责领导中国共产党和中国政府,并监督中国的政治和经济。
User:我们在聊谁呢？
Assistant:习近平是中国共产党总书记,也是中国国家主席。<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>


In [28]:
model.save_pretrained("lora-alpaca")